# Исследовательский анализ

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)
pd.set_option("display.precision",6)
pd.set_option("plotting.backend", "matplotlib")

#Для корректного отображения графиков в ноутбуках используется команда %matplotlib inline
%matplotlib inline
# для корректного отображения графиков на тёмном фоне
plt.style.use('default')

# Загрузка и анализ сырых данных

In [2]:
project_path = os.path.abspath(os.path.join(os.getcwd(), os.path.pardir, os.path.pardir))
f_input = os.path.join("data", "raw", "SF_Mandarin_dataset_ver3_csv.csv")
filename = os.path.join(project_path,f_input)

In [3]:
df = pd.read_csv(filename, sep=';')

Размер набора данных

In [4]:
print(f'Строк - {df.shape[0]}\nСтолбцов - {df.shape[1]}')

Строк - 8787
Столбцов - 22


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8787 entries, 0 to 8786
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SkillFactory_Id    6603 non-null   float64
 1   BirthDate          6603 non-null   object 
 2   education          6603 non-null   object 
 3   employment status  6603 non-null   object 
 4   Value              6523 non-null   object 
 5   JobStartDate       6494 non-null   object 
 6   Position           6494 non-null   object 
 7   MonthProfit        6603 non-null   float64
 8   MonthExpense       6603 non-null   float64
 9   Gender             6566 non-null   float64
 10  Family status      6566 non-null   object 
 11  ChildCount         6566 non-null   float64
 12  SNILS              6603 non-null   float64
 13  BankA_decision     6603 non-null   object 
 14  BankB_decision     6603 non-null   object 
 15  BankC_decision     6603 non-null   object 
 16  BankD_decision     6603 

In [6]:
df.describe()

,SkillFactory_Id,MonthProfit,MonthExpense,Gender,ChildCount,SNILS,Merch_code,Loan_amount,Loan_term
count,6603.000000,6.603000e+03,6603.000000,6566.000000,6566.000000,6603.000000,6603.000000,6603.000000,6603.000000
mean,4388.192640,1.484528e+05,26234.063153,0.309625,0.097320,0.350295,38.794184,76362.108133,12.117219
std,2515.520596,7.742496e+04,22557.993167,0.462374,0.423026,0.477099,23.929860,50978.500496,5.996582
min,1.000000,5.000000e+01,0.000000,0.000000,0.000000,0.000000,1.000000,4000.000000,6.000000
25%,2225.500000,1.000000e+05,15000.000000,0.000000,0.000000,0.000000,18.000000,35000.000000,6.000000
50%,4431.000000,1.450000e+05,22500.000000,0.000000,0.000000,0.000000,36.000000,68000.000000,12.000000
75%,6516.500000,1.900000e+05,31000.000000,1.000000,0.000000,1.000000,63.000000,101000.000000,18.000000
max,8787.000000,1.200000e+06,500000.000000,1.000000,5.000000,1.000000,80.000000,200000.000000,24.000000


In [7]:
# Количество дубликатов:
df.duplicated().sum()

2183

In [8]:
# Строк с пустыми значениями
df[df.isnull().any(axis=1)].shape[0]

2323

# Загрузка и анализ данных после предобработки

## Данные после 1-го этапа заполнения пропусков

In [9]:
# f_input = os.path.join("data", "raw", "SF_Mandarin_dataset_ver3_csv.csv")
f_input1="data/stage_fill_na/dataset.csv"
filename = os.path.join(project_path,f_input1)
df1 = pd.read_csv(filename, sep=';')

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603 entries, 0 to 6602
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SkillFactory_Id    6603 non-null   float64
 1   BirthDate          6603 non-null   object 
 2   education          6603 non-null   object 
 3   employment status  6603 non-null   object 
 4   Value              6603 non-null   object 
 5   JobStartDate       6603 non-null   object 
 6   Position           6494 non-null   object 
 7   MonthProfit        6603 non-null   float64
 8   MonthExpense       6603 non-null   float64
 9   Gender             6603 non-null   float64
 10  Family status      6603 non-null   object 
 11  ChildCount         6603 non-null   float64
 12  SNILS              6603 non-null   float64
 13  BankA_decision     6603 non-null   object 
 14  BankB_decision     6603 non-null   object 
 15  BankC_decision     6603 non-null   object 
 16  BankD_decision     6603 

## Данные после 2-го этапа корректировки типов данных

In [11]:
f_input2="data/stage_data_prepare/dataset.csv"
filename = os.path.join(project_path,f_input2)
df2 = pd.read_csv(filename, sep=';', parse_dates=['JobStartDate', 'BirthDate'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603 entries, 0 to 6602
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SkillFactory_Id    6603 non-null   int64         
 1   BirthDate          6603 non-null   datetime64[ns]
 2   education          6603 non-null   object        
 3   employment status  6603 non-null   object        
 4   Value              6603 non-null   object        
 5   JobStartDate       6603 non-null   datetime64[ns]
 6   Position           6494 non-null   object        
 7   MonthProfit        6603 non-null   float64       
 8   MonthExpense       6603 non-null   float64       
 9   Gender             6603 non-null   int64         
 10  Family status      6603 non-null   object        
 11  ChildCount         6603 non-null   int64         
 12  SNILS              6603 non-null   int64         
 13  BankA_decision     6603 non-null   object        
 14  BankB_de

In [12]:
# Количество дубликатов:
df2.duplicated().sum()

0

In [13]:
# Строк с пустыми значениями
df2[df2.isnull().any(axis=1)].shape[0]

109

In [56]:
f_input3="data/stage_fix_errors/dataset.csv"
filename = os.path.join(project_path,f_input3)
df3 = pd.read_csv(filename, sep=';', parse_dates=['JobStartDate', 'BirthDate'])
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603 entries, 0 to 6602
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SkillFactory_Id    6603 non-null   int64         
 1   BirthDate          6603 non-null   datetime64[ns]
 2   education          6603 non-null   object        
 3   employment status  6603 non-null   object        
 4   Value              6544 non-null   object        
 5   JobStartDate       6603 non-null   object        
 6   Position           6494 non-null   object        
 7   MonthProfit        6603 non-null   float64       
 8   MonthExpense       6603 non-null   float64       
 9   Gender             6603 non-null   int64         
 10  Family status      6603 non-null   object        
 11  ChildCount         6603 non-null   int64         
 12  SNILS              6603 non-null   int64         
 13  BankA_decision     6603 non-null   object        
 14  BankB_de

### Проверка методов предобработки

In [14]:
from importlib import reload
import data_prepare
reload(data_prepare)
df2t = data_prepare.prepare_dataset(df1)
df2t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603 entries, 0 to 6602
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SkillFactory_Id    6603 non-null   int64         
 1   BirthDate          6603 non-null   datetime64[ns]
 2   education          6603 non-null   object        
 3   employment status  6603 non-null   object        
 4   Value              6603 non-null   object        
 5   JobStartDate       6603 non-null   datetime64[ns]
 6   Position           6494 non-null   object        
 7   MonthProfit        6603 non-null   float64       
 8   MonthExpense       6603 non-null   float64       
 9   Gender             6603 non-null   int64         
 10  Family status      6603 non-null   object        
 11  ChildCount         6603 non-null   int64         
 12  SNILS              6603 non-null   int64         
 13  BankA_decision     6603 non-null   object        
 14  BankB_de

In [40]:
from importlib import reload
import fix_errors
reload(fix_errors)

<module 'fix_errors' from '/mnt/data/projects/active/urfu/hacaton2/scripts/data_scripts/fix_errors.py'>

In [16]:
df2t['Family status'][0]

'Никогда в браке не состоял(а)'

In [17]:
fix_errors.check_family_status(df2t['Family status'][0])

False

In [18]:
a=df2t.iloc[1201, :]
a

SkillFactory_Id                                            1627
BirthDate                                   1995-07-03 00:00:00
education                              Среднее профессиональное
employment status    Работаю по найму полный рабочий день/служу
Value                                                3 - 4 года
                                        ...                    
BankE_decision                                          success
Merch_code                                                 15.0
Loan_amount                                             80000.0
Loan_term                                                  12.0
Goods_category                                 Medical_services
Name: 1201, Length: 22, dtype: object

In [19]:
fix_errors.fix_expense(a)
a

/mnt/data/projects/active/urfu/hacaton2/scripts/data_scripts/fix_errors.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_data['MonthExpense'] = real_expense


SkillFactory_Id                                            1627
BirthDate                                   1995-07-03 00:00:00
education                              Среднее профессиональное
employment status    Работаю по найму полный рабочий день/служу
Value                                                3 - 4 года
                                        ...                    
BankE_decision                                          success
Merch_code                                                 15.0
Loan_amount                                             80000.0
Loan_term                                                  12.0
Goods_category                                 Medical_services
Name: 1201, Length: 22, dtype: object

In [23]:
display(a['Value'])
display(type(a['Value']))
display(fix_errors.values_to_num(a['Value']))

'3 - 4 года'

str

36

In [21]:
display(a['JobStartDate'])
display(fix_errors.time_diff(a['JobStartDate'])/12)

Timestamp('2019-05-01 00:00:00')

4.666666666666667

In [50]:
# b=df2t.iloc[5457, :]
b=df2t.iloc[4348, :]
b

SkillFactory_Id                                            5769
BirthDate                                   2005-02-04 00:00:00
education                                  Неоконченное среднее
employment status    Работаю по найму полный рабочий день/служу
Value                                         6 месяцев - 1 год
                                        ...                    
BankE_decision                                          success
Merch_code                                                 43.0
Loan_amount                                            196000.0
Loan_term                                                  24.0
Goods_category                                           Travel
Name: 4348, Length: 22, dtype: object

In [51]:
total_seniority = int(fix_errors.values_to_num(b['Value']))
total_seniority

6

In [52]:
from dateutil.relativedelta import relativedelta
from datetime import datetime
age = relativedelta(datetime.today(), b['BirthDate'])
display(age)
display(age.months + age.years * 12)
age.months + (age.years - 16) * 12

relativedelta(years=+18, months=+10, days=+4, hours=+4, minutes=+18, seconds=+5, microseconds=+821186)

226

34

In [53]:
last_seniority = relativedelta(datetime.today(), b['JobStartDate'])
display(last_seniority)
last_seniority.months + last_seniority.years * 12

relativedelta(years=+18, months=+9, days=+7, hours=+4, minutes=+18, seconds=+16, microseconds=+480731)

225

In [54]:
c=fix_errors.fix_seniority(b)
c

/mnt/data/projects/active/urfu/hacaton2/scripts/data_scripts/fix_errors.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  :param str_value:  Данные из заявки
/mnt/data/projects/active/urfu/hacaton2/scripts/data_scripts/fix_errors.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


SkillFactory_Id                                            5769
BirthDate                                   2005-02-04 00:00:00
education                                  Неоконченное среднее
employment status    Работаю по найму полный рабочий день/служу
Value                                                2 - 3 года
                                        ...                    
BankE_decision                                          success
Merch_code                                                 43.0
Loan_amount                                            196000.0
Loan_term                                                  24.0
Goods_category                                           Travel
Name: 4348, Length: 22, dtype: object

In [56]:
df2['education'].value_counts()

education
Высшее - специалист         5296
Среднее профессиональное     471
Среднее                      233
Неоконченное высшее          233
Бакалавр                     153
Несколько высших             110
Магистр                       73
Неоконченное среднее          30
MBA                            3
Ученая степень                 1
Name: count, dtype: int64

In [57]:
df1['employment status'].value_counts()

employment status
Работаю по найму полный рабочий день/служу    4858
Собственное дело                              1527
Не работаю                                      75
Работаю по найму неполный рабочий день          62
Студент                                         44
Декретный отпуск                                19
Пенсионер                                       18
Name: count, dtype: int64

In [58]:
df1['Value'].value_counts()

Value
10 и более лет       3615
3 - 4 года            528
2 - 3 года            503
4 - 5 лет             371
5 - 6 лет             293
1 - 2 года            279
6 - 7 лет             250
7 - 8 лет             225
8 - 9 лет             152
6 месяцев - 1 год     127
9 - 10 лет             88
Нет стажа              80
4 - 6 месяцев          56
менее 4 месяцев        36
Name: count, dtype: int64

In [59]:
df1['Family status'].value_counts()

Family status
Никогда в браке не состоял(а)               4474
Женат / замужем                             1490
Разведён / Разведена                         486
Гражданский брак / совместное проживание     104
Вдовец / вдова                                49
Name: count, dtype: int64

In [60]:
df1['Goods_category'].value_counts()

Goods_category
Furniture           976
Mobile_devices      957
Travel              956
Medical_services    940
Education           936
Fitness             928
Other               910
Name: count, dtype: int64

In [61]:
df1['Merch_code'].value_counts()

Merch_code
35.0    110
11.0    110
74.0    108
62.0    108
63.0    106
       ... 
19.0     84
36.0     83
6.0      82
72.0     79
66.0     79
Name: count, Length: 70, dtype: int64

In [62]:
df1['BankA_decision'].value_counts()

BankA_decision
success    4970
denied     1562
error        71
Name: count, dtype: int64